# Food, Agriculture, and Soils: California Legislation

This notebook explores state level legislation from the Legiscan API. It conducts text analysis of all available California Legislation that mentions food and agriculture from years 20XX to 2023, with special attention to mentions of land, soil, and environmental management.

LegiScan API documentation [here](https://legiscan.com/gaits/documentation/legiscan) and [here](https://api.legiscan.com/dl/).

In [1]:
# libraries
import pandas as pd
import geopandas as gpd
import numpy as np

import requests
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup

import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import json

# set display
pd.options.display.max_columns = 150
pd.options.display.max_rows = 300

First, I will pull the bill ID, year/session, and full texts for all CA legisation that passed mentioning "food" and "agriculture". 

Then I will narrow my search to all bills mentioning "food", "agriculture", "environment", and "soil".

I will store relevant information into a dictionary that can be transformed into a pandas dataframe.

Pull API template: "https://api.legiscan.com/?key=APIKEY&op=OPERATION&PARAMS"
APIKey = '7e00040f1f7618af234e7415484d2494'

**OPERATIONS**
getBill, getBillText, getSearch, getSearchRaw

**PARAMETERS**
state, year, query (URL encoded), page

**IDENTIFIERS**
bill_id, doc_id

In [3]:
# request: CA, food and ag, results for current year
requeststring = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=2&query=food%20and%20agriculture"

In [7]:
# request: CA, food and ag, results for recent years
requeststring1 = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=3&query=food%20and%20agriculture"

In [5]:
# request: CA, food and ag, results for all available years
requeststring2 = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=1&query=food%20and%20agriculture"

In [8]:
# pull request: getting results as JSON object
r = requests.get(requeststring1)

In [17]:
# examining results
#print(r.text)

In [9]:
# turning JSON object into a dict
dict1 = json.loads(r.text)

In [10]:
# examining structure of dictionary and identifying relevant keys
print(dict1.keys())
print(dict1['searchresult'].keys())
print(dict1['searchresult']['summary'].keys())
print(dict1['searchresult']['results'])

dict_keys(['status', 'searchresult'])
dict_keys(['summary', 'results'])
dict_keys(['page', 'range', 'relevancy', 'count', 'page_current', 'page_total', 'query'])
[{'relevance': 100, 'bill_id': 1453447, 'change_hash': 'c4cbd2bf405a5c9f9d8db0604c01ee91'}, {'relevance': 99, 'bill_id': 1693920, 'change_hash': '2cc2c16e46855bab164790df5a2dc955'}, {'relevance': 99, 'bill_id': 1456707, 'change_hash': '0f15fc913ea7989404f0e73f48c7b1de'}, {'relevance': 99, 'bill_id': 1388510, 'change_hash': '75ed87f9287e2be9b2670e33d0fb03c5'}, {'relevance': 99, 'bill_id': 1711942, 'change_hash': 'cd9d825f6d643ff19d5d789db7760174'}, {'relevance': 99, 'bill_id': 1454994, 'change_hash': '41e3c3667fa12d949ffa64b377696f0e'}, {'relevance': 99, 'bill_id': 1594832, 'change_hash': '9fce132981f80e04c7d4e45c8b4c4a04'}, {'relevance': 99, 'bill_id': 1614129, 'change_hash': '47b91c807bf15b8e60c9a568bbf154b3'}, {'relevance': 99, 'bill_id': 1592497, 'change_hash': '53adde10972d4112d157cbd0edb0f15f'}, {'relevance': 99, 'bill_id

In [11]:
# creating subset dict with only keys of interest
list1 = dict1['searchresult']['results']
list1[:5]

[{'relevance': 100,
  'bill_id': 1453447,
  'change_hash': 'c4cbd2bf405a5c9f9d8db0604c01ee91'},
 {'relevance': 99,
  'bill_id': 1693920,
  'change_hash': '2cc2c16e46855bab164790df5a2dc955'},
 {'relevance': 99,
  'bill_id': 1456707,
  'change_hash': '0f15fc913ea7989404f0e73f48c7b1de'},
 {'relevance': 99,
  'bill_id': 1388510,
  'change_hash': '75ed87f9287e2be9b2670e33d0fb03c5'},
 {'relevance': 99,
  'bill_id': 1711942,
  'change_hash': 'cd9d825f6d643ff19d5d789db7760174'}]

In [12]:
# generating list of bill_ids
# https://www.geeksforgeeks.org/python-get-values-of-particular-key-in-list-of-dictionaries/
bills = [value['bill_id'] for value in list1]
bills[:5]

[1453447, 1693920, 1456707, 1388510, 1711942]

In [13]:
# turning list of bill_ids for each relevant result from query into a df
#df = pd.DataFrame(bills, columns = ['bill_id'])
#df.head()

In [14]:
# sampling bill pull
bill = 'https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id=1456707'
billa = 'https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id=1592779'
b = requests.get(bill)
ba = requests.get(billa)
binfo = json.loads(b.text)
binfoa = json.loads(ba.text)
print(binfo.keys()) 
print(binfoa.keys()) 
#print(binfo)
print(binfoa)

dict_keys(['status', 'bill'])
dict_keys(['status', 'alert'])
{'status': 'ERROR', 'alert': {'message': 'Unknown bill id'}}


In [160]:
# exploring structure and extracting doc_ids from bill: BILL
print(binfo.keys())
print(binfo['bill'].keys())
print(binfo['bill']['texts'])

dict_keys(['status', 'bill'])
dict_keys(['bill_id', 'change_hash', 'session_id', 'session', 'url', 'state_link', 'completed', 'status', 'status_date', 'progress', 'state', 'state_id', 'bill_number', 'bill_type', 'bill_type_id', 'body', 'body_id', 'current_body', 'current_body_id', 'title', 'description', 'pending_committee_id', 'committee', 'referrals', 'history', 'sponsors', 'sasts', 'subjects', 'texts', 'votes', 'amendments', 'supplements', 'calendar'])
[{'doc_id': 2704856, 'date': '2023-02-16', 'type': 'Introduced', 'type_id': 1, 'mime': 'text/html', 'mime_id': 1, 'url': 'https://legiscan.com/CA/text/AB1197/id/2704856', 'state_link': 'https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202320240AB1197#99INT', 'text_size': 7488, 'text_hash': 'b048ef4ee2d51bcb0266641e75d77798', 'alt_bill_text': 0, 'alt_mime': '', 'alt_mime_id': 0, 'alt_state_link': '', 'alt_text_size': 0, 'alt_text_hash': ''}, {'doc_id': 2743399, 'date': '2023-03-13', 'type': 'Amended', 'type_id': 3,

In [175]:
# exploring structure and extracting doc_ids from bill: ALERT
print(binfoa.keys())
print(binfoa['alert'].keys())
print(binfoa['status'])
print(binfoa['alert']['message'])

dict_keys(['status', 'alert'])
dict_keys(['message'])
ERROR
Unknown bill id


In [15]:
# isolating list of dictionaries
doclist = binfo['bill']['texts']

In [16]:
# generating list of bill_ids
docs = [sub['doc_id'] for sub in doclist]
docs

[2299860, 2352258, 2430109, 2431702, 2432952, 2436333]

In [17]:
# indexing last element (most recently amended version of bill text)
docs[-1]

2436333

In [18]:
# sampling bill text pull
btext1 = 'https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBillText&id=2436333'
b = requests.get(btext1)
btext = json.loads(b.text)
print(btext.keys()) 
print(btext)

dict_keys(['status', 'text'])
{'status': 'OK', 'text': {'doc_id': 2436333, 'bill_id': 1456707, 'date': '2021-10-07', 'type': 'Chaptered', 'type_id': 6, 'mime': 'text/html', 'mime_id': 1, 'url': 'https://legiscan.com/CA/text/AB1009/id/2436333', 'state_link': 'https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202120220AB1009#94CHP', 'text_size': 16842, 'text_hash': '210a902acf5cad5b828a388d0c45a676', 'doc': '...', 'alt_bill_text': 0, 'alt_mime': '', 'alt_mime_id': 0, 'alt_state_link': '', 'alt_text_size': 0, 'alt_text_hash': '', 'alt_doc': ''}}


In [19]:
# exploring structure
print(btext.keys())
print(btext['text'].keys())
print(btext['text']['url'])
print(btext['text']['state_link']) # this URL less likely to turn up errors

dict_keys(['status', 'text'])
dict_keys(['doc_id', 'bill_id', 'date', 'type', 'type_id', 'mime', 'mime_id', 'url', 'state_link', 'text_size', 'text_hash', 'doc', 'alt_bill_text', 'alt_mime', 'alt_mime_id', 'alt_state_link', 'alt_text_size', 'alt_text_hash', 'alt_doc'])
https://legiscan.com/CA/text/AB1009/id/2436333
https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202120220AB1009#94CHP


In [20]:
# extracting status info from leginfo
statusURL = btext['text']['state_link'].replace('Text', 'Status')
statusURL1 = btext['text']['url'].split("/id")[0].replace('text', 'bill')
print(statusURL)
print(statusURL1)

https://leginfo.legislature.ca.gov/faces/billStatusClient.xhtml?bill_id=202120220AB1009#94CHP
https://legiscan.com/CA/bill/AB1009


In [21]:
# extracting bill info and text from bill text url
URL = btext['text']['state_link']
URLa = btext['text']['url']
#r = urlopen(URLa)
#html_bytes = r.read()
#html = html_bytes.decode("utf-8")
#soup = BeautifulSoup(html, features='html.parser')

# extracting status from state bill link
URL1 = statusURL
r1 = urlopen(URL1)
html_bytes1 = r1.read()
html1 = html_bytes1.decode("utf-8")
soup1 = BeautifulSoup(html1, features='html.parser')

# extracting status from LegiScan link
URL2 = statusURL1
#r2 = urlopen(URL2)
#html_bytes2 = r2.read()
#html2 = html_bytes2.decode("utf-8")
#soup2 = BeautifulSoup(html2, features='html.parser')

In [188]:
# inspecting structure to locate text bloc
print(soup.prettify())

<?xml version='1.0' encoding='UTF-8' ?>
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head id="j_idt5">
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!--&lt;meta http-equiv="refresh" content=";url=faces/home.xhtml"/&gt;-->
  <link href="https://leginfo.legislature.ca.gov/resources/css/leginfo_master.css" media="screen   ,print" rel="stylesheet" type="text/css"/>
  <!--&lt;link rel="stylesheet" type="text/css" href="./resources/css/leginfo_mobile.css" media="screen and (-webkit-min-device-pixel-ratio: 1.1) and (max-device-width: 990px)" /&gt;
    &lt;link rel="stylesheet" type="text/css" href="./resources/css/leginfo_mobile.css" media="screen and (min-device-width: 481px)and (max-device-width: 640px)and (-webkit-max-device-pixel-ratio: 1)" /&gt;-->
  <link href="https://leginfo.legislature.ca.gov/resources/css/leginfo_mobile.css" media="screen and (orient

In [32]:
# main page: locating title in html
t = soup.find_all('title')
print(len(t))
print(t[0])

2
<title>Bill Text  - AB-1197 Agricultural Protection Planning Grant Program: local food producers.</title>


In [33]:
# main page: locating bill year introduced in html
#y = soup.find_all(id = 'bill_intro_date')
y = soup.find_all(class_ = 'bill_title_yr')
print(len(y))
#print(y[0])
print(y)

1
[<span class="bill_title_yr">(2023-2024)</span>]


In [243]:
# main page: locating bill year amended in html
#y1 = soup.find_all('tbody')
#print(len(y1))
#print(type(y1))
#print(y1[0])

In [25]:
# main page: locating bill text bloc in html
ti = soup.find_all('div', id = 'bill')
print(len(ti))
print(type(ti))
print(ti)

1
<class 'bs4.element.ResultSet'>
[<div id="bill"><div style=" text-transform: uppercase"><h2 style=" text-align: left;">The people of the State of California do enact as follows:</h2><br/></div><div id="s10.8987986890852493"><div><div style="margin:0 0 1em 0;"><h3 style="font-weight:bold; display:inline;">SECTION 1.</h3> The Legislature finds and declares all of the following:</div><div style="margin:0 0 1em 0;">(a) Over one-third of the country’s vegetables and two-thirds of the country’s fruits and nuts are grown in California. California’s agricultural abundance includes more than 400 commodities. California is the leading state in the nation for cash farm receipts, accounting for over 13 percent of the nation’s total agricultural value.</div><div style="margin:0 0 1em 0;">(b) California is the sole producer (99 percent or more) in the nation of the following crops: almonds, artichokes, dates, figs, garlic, grapes, raisins, kiwi, melons, olives, clingstone peaches, pistachios,
				

In [47]:
# status page: locating bill status: STATE link
#status = soup.find_all('tr')
#status = soup.find_all('td')
#status = soup1.find_all('td', class_= "statusLabelData")
#status = soup1.find_all('td', scope = "row")
status = soup1.find_all(id = "billhistory")
print(len(status))
print(type(status))
print(status)
#print(status.pop(0))

1
<class 'bs4.element.ResultSet'>
[<table cellspacing="0" id="billhistory" width="100%">
<thead>
<tr>
<th colspan="2" scope="colgroup">Last 5 History Actions</th>
</tr>
<tr>
<th class="status_col_width" scope="col">Date</th>
<th scope="col">Action</th>
</tr>
</thead>
<tbody>
<tr>
<td scope="row">07/05/23</td>
<td>From committee: Do pass and re-refer to Com. on  APPR. (Ayes  3. Noes  0.) (July  3). Re-referred to Com. on  APPR.</td>
</tr>
<tr>
<td scope="row">06/26/23</td>
<td>Read second time and amended.  Re-referred to Com. on  AGRI.</td>
</tr>
<tr>
<td scope="row">06/22/23</td>
<td>From committee: Amend, and do pass as amended and re-refer to Com. on  AGRI. (Ayes  9. Noes  0.) (June  22).</td>
</tr>
<tr>
<td scope="row">06/12/23</td>
<td>From committee chair, with author's amendments:  Amend, and re-refer to committee.  Read second time, amended, and re-referred to Com. on  N.R. &amp; W.</td>
</tr>
<tr>
<td scope="row">06/07/23</td>
<td>Referred to Coms. on  N.R. &amp; W. and  AGRI.

In [337]:
# status page: locating bill status: LEGISCAN main link
status = soup.find_all('b')
status = soup.find_all('hr')
status = soup.find_all(id = "gaits-social")
status = soup.find_all('span', class_ = 'tab')
print(len(status))
print(type(status))
print(status)
#print(status[-1])
#print(status.pop())

1
<class 'bs4.element.ResultSet'>
[<div id="bill-last-action" style="margin: 0 1em">Spectrum: Partisan Bill (Democrat 1-0)<br/>Status: Engrossed on May 30 2023 - 50% progression<br/>Action: 2023-07-13 - From committee: Do pass as amended and re-refer to Com. on APPR. (Ayes 6. Noes 1.) (July 11).<br/>Pending: <a href="/CA/pending/assembly-appropriations-committee/id/449" title="View other bills referred to CA Assembly Appropriations Committee">Assembly Appropriations Committee</a><br/>Text: <a href="/CA/text/SB485/2023" title="View latest bill text for California SB485">Latest bill text (Amended) [HTML]</a></div>]


In [337]:
# status page: locating bill status: LEGISCAN bill page link
#status = soup.find_all('tr')
#status = soup.find_all('td')
status = soup2.find_all(id = "bill-last-action")
print(len(status))
print(type(status))
print(status)
#print(status[-1])
#print(status.pop())

1
<class 'bs4.element.ResultSet'>
[<div id="bill-last-action" style="margin: 0 1em">Spectrum: Partisan Bill (Democrat 1-0)<br/>Status: Engrossed on May 30 2023 - 50% progression<br/>Action: 2023-07-13 - From committee: Do pass as amended and re-refer to Com. on APPR. (Ayes 6. Noes 1.) (July 11).<br/>Pending: <a href="/CA/pending/assembly-appropriations-committee/id/449" title="View other bills referred to CA Assembly Appropriations Committee">Assembly Appropriations Committee</a><br/>Text: <a href="/CA/text/SB485/2023" title="View latest bill text for California SB485">Latest bill text (Amended) [HTML]</a></div>]


In [246]:
# remove html style and tags from title
for text in t[0]:
    title = text.text.strip()
    title = title.split("- ")[-1]
title

'SB-485 Elections: election worker protections.'

In [28]:
# remove html style and tags from year int
for text in y:
    year = text.text.strip()
    #yeari = yeari[-4:]
year

'(2021-2022)'

In [249]:
# remove html style and tags from year amended
#for text in y1[0]:
 #   yeara = text.text.strip()
 #   yeara = yeara[-4:]
#yeara

In [250]:
# remove html style and tags from bill text
for text in ti:
    
    billtxt = text.text.strip()#.encode("ascii", "ignore")
    #billtxt = str(billtxt)
    
    # remove non ASCII chars
    # https://stackoverflow.com/questions/20078816/replace-non-ascii-characters-with-a-single-space
    billtxt = re.sub(r'[^\x00-\x7F]+', ' ', billtxt)#.replace('\n\t\t\t\t\t\t', ' ')
    
    # remove tabs and new lines
    billtxt = re.sub(r'(\s)', ' ', billtxt)

    # add space after colons and semicolons
    billtxt = re.sub(r'(\:|\;)+', r'\1 ', billtxt)
    
    # add space after periods only if preceded by words
    billtxt = re.sub(r'([A-z]{4,}\.)+', r'\1 ', billtxt)
    
    # remove any extra white spaces
    billtxt = re.sub(r'\s+', ' ', billtxt)
    
    # testing search
    #comp = re.compile(r'([A-z]{4,}\.)+')
   # find = re.search(comp, billtxt)

#find
print(type(billtxt))
billtxt

<class 'str'>


'The people of the State of California do enact as follows: SECTION 1. Section 18502 of the Elections Code is amended to read: 18502. (a) Any person who in any manner interferes with any of the following as to prevent the election or canvass from being fairly held and lawfully conducted, is punishable by imprisonment pursuant to subdivision (h) of Section 1170 of the Penal Code for 16 months or two or three years: (1) An elections official or their staff, including temporary workers and volunteers, in their performance of any duty related to conducting an election or canvass. (2) A member of a precinct board, in their performance of any duty related to assisting with conducting an election or canvass. (3) A voter lawfully exercising their rights of voting at an election. (b) For purposes of this section, conducting an election or canvass includes the election observation process governed by this code and applicable regulations adopted by the Secretary of State. (c) For purposes of this

In [43]:
# remove html style and tags from bill status: State link
for text in status:
    status = text.text
    status = status.split("Action")[-1]
    status = re.sub(r'(\s)', ' ', status)
    status = re.sub(r'\s+', ' ', status)
    
status

" 07/05/23 From committee: Do pass and re-refer to Com. on APPR. (Ayes 3. Noes 0.) (July 3). Re-referred to Com. on APPR. 06/26/23 Read second time and amended. Re-referred to Com. on AGRI. 06/22/23 From committee: Amend, and do pass as amended and re-refer to Com. on AGRI. (Ayes 9. Noes 0.) (June 22). 06/12/23 From committee chair, with author's amendments: Amend, and re-refer to committee. Read second time, amended, and re-referred to Com. on N.R. & W. 06/07/23 Referred to Coms. on N.R. & W. and AGRI. "

In [319]:
# remove html style and tags from bill status: LegiScan link
for text in status:
    status = text.text.strip()
    status = status.split("Action: ")[-1].split("Pending")[0]

status

'2023-07-13 - From committee: Do pass as amended and re-refer to Com. on APPR. (Ayes 6. Noes 1.) (July 11).'

In [68]:
# stripping lingering unicode characters
billtxt1 = billtxt.astype('str')
billtxt1

AttributeError: 'str' object has no attribute 'astype'

In [122]:
# creating bill_id
bill_id = title.split("- ")[-1][:7]

# concatenating into one list
infotxt1 = [bill_id, title, yeari, status, billtxt]
infotxt1

['SB-118 ',
 'SB-118 Budget Act of 2023: health.',
 '2023',
 'Referred to Coms. on  N.R. & W. and  AGRI.',
 'The people of the State of California do enact as follows:SECTION 1.\xa0Section 1295 of the Code of Civil Procedure is amended to read:1295.\xa0(a)\xa0Any contract for medical services which contains a provision for arbitration of any dispute as to professional negligence of a health care provider shall have such provision as the first article of the contract and shall be expressed in the following language: “It is understood that any dispute as to medical malpractice, that is as to whether any medical services rendered under this contract were unnecessary or unauthorized or were improperly, negligently or incompetently rendered, will be determined by submission to arbitration as provided by California law, and not by a lawsuit or resort to court process except as California law provides for judicial review of arbitration proceedings. Both parties to this contract, by entering i

In [102]:
# remove html style and tags from all bill info (B)

# list of all bill elements
billinfo = [t[0], y, y1[0], ti]

# remove html style and tags from bill text
infotxt = []

for info in billinfo:
    for text in info:
        row = text.text.strip()
        infotxt.append(row)
        
    # create bill id from bill title
    bill_id = infotxt[0].split("- ")[-1][:7]
    
# add an item to the list that will serve as the bill id later
infotxt.insert(0, bill_id)
        
print(len(infotxt))
infotxt

5


['AB-1197',
 'Bill Text  - AB-1197 Agricultural Protection Planning Grant Program: local food producers.',
 'February\xa016,\xa02023',
 'Amended\n              \xa0IN\xa0\n                Senate\n              \xa0June\xa026,\xa02023',
 'The people of the State of California do enact as follows:SECTION 1.\xa0Section 10280 of the Public Resources Code is amended to read:10280.\xa0The Agricultural Protection Planning Grant Program is hereby established within the Department of Conservation, to provide planning grants to do all of the following:(a)\xa0Conserve California’s most productive farmlands and ecologically important rangelands.(b)\xa0Advance California’s climate change goals through carbon sequestration and greenhouse gas emissions reductions resulting from the implementation of local plans.(c)\xa0Maintain local food supplies, local food producers, and agricultural economies through the protection of agricultural lands.SEC. 2.\xa0Section 10280.5 of the Public Resources Code is am

In [104]:
# storing bill info in df    
itxtdf = pd.DataFrame(infotxt1).transpose()

# rename columns
#itxtdf.rename(columns = {'0':'title', '1':'y_intro', '2': 'y_recent', '3':'text'}, inplace = True)
itxtdf.columns = ['bill_id', 'title', 'y_intro', 'y_recent', 'text']

# optional: create column to serve as new index ID?
#itxtdf.set_index('bill_id', inplace = True)

itxtdf

,bill_id,title,y_intro,y_recent,text
0,AB-1197,AB-1197 Agricultural Protection Planning Grant...,2023,2023,The people of the State of California do enact...


In [68]:
# further processing bill text for NLP

# list of stopwords to exclude
swords = stopwords.words('english')

# stripping nonessential characters
textonly = re.sub(r"[^A-z\s]", "", billtxt)

# turning cleaned bill text into list of words
wordlist = [word for word in word_tokenize(textonly.lower()) 
                 if word not in swords]
print(len(wordlist))
wordlist

641


['people',
 'state',
 'california',
 'enact',
 'followssection',
 'section',
 'public',
 'resources',
 'code',
 'amended',
 'read',
 'agricultural',
 'protection',
 'planning',
 'grant',
 'program',
 'hereby',
 'established',
 'within',
 'department',
 'conservation',
 'provide',
 'planning',
 'grants',
 'followinga',
 'conserve',
 'californias',
 'productive',
 'farmlands',
 'ecologically',
 'important',
 'rangelandsb',
 'advance',
 'californias',
 'climate',
 'change',
 'goals',
 'carbon',
 'sequestration',
 'greenhouse',
 'gas',
 'emissions',
 'reductions',
 'resulting',
 'implementation',
 'local',
 'plansc',
 'maintain',
 'local',
 'food',
 'supplies',
 'local',
 'food',
 'producers',
 'agricultural',
 'economies',
 'protection',
 'agricultural',
 'landssec',
 'section',
 'public',
 'resources',
 'code',
 'amended',
 'read',
 'following',
 'terms',
 'following',
 'meanings',
 'used',
 'division',
 'unless',
 'context',
 'clearly',
 'requires',
 'otherwisea',
 'authority',
 'means'

## Putting All the Pieces Together

This code aims to:

1. Extract the individual *bill_id* for each bill in the search results for a query of "food and agriculture" bills in California for the most recent legislative session;
2. extract the *doc_id* for the most recently amended version of each bill;
3. extract (A) the bill title and (B) the bill text for each document;
4. and finally, clean the bill text (prepare into either (A) a relatively clean string text block or (B) a cleaned wordlist).

In [3]:
## SCRAPE SEARCH RESULTS for bill_ids

# scrape LegiScan search results for CA bills RE: food and ag for current year
requeststring = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=2&query=food%20and%20agriculture"
# request: CA, food and ag, results for recent years
requeststring1 = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=3&query=food%20and%20agriculture"
# request: CA, food and ag, results for all available years
requeststring2 = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=1&query=food%20and%20agriculture"

# pull request: ge\tting results as JSON object
r = requests.get(requeststring1)
# turning JSON object into a dict
dict1 = json.loads(r.text)

# creating subset dict with only keys of interest
list1 = dict1['searchresult']['results']
# generating list of bill_ids
bills = [value['bill_id'] for value in list1]

# each bill in bills is stored as an integer; convert to list of str
bills1 = [str(bill) for bill in bills]

#inspect/show results
print(len(bills1))
bills1[:5]

565


['1453447', '1693920', '1456707', '1388510', '1711942']

In [166]:
# TEST
docs = []
docs1 = {}

for bill in bills1:
    
    # extract bill info
    bill_id = bill
    requestbill = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id="+bill_id
    b = requests.get(requestbill)
    binfo = json.loads(b.text)
    
    # FIND doc_id for most recent bill version
    docs1[bill] = binfo.keys()
    
docs1

{'1453447': dict_keys(['status', 'bill']),
 '1693920': dict_keys(['status', 'bill']),
 '1456707': dict_keys(['status', 'bill']),
 '1388510': dict_keys(['status', 'bill']),
 '1711942': dict_keys(['status', 'bill']),
 '1454994': dict_keys(['status', 'bill']),
 '1594832': dict_keys(['status', 'bill']),
 '1614129': dict_keys(['status', 'bill']),
 '1592497': dict_keys(['status', 'bill']),
 '1453388': dict_keys(['status', 'bill']),
 '1593665': dict_keys(['status', 'bill']),
 '1702780': dict_keys(['status', 'bill']),
 '1458834': dict_keys(['status', 'bill']),
 '1433188': dict_keys(['status', 'bill']),
 '1451034': dict_keys(['status', 'bill']),
 '1578060': dict_keys(['status', 'bill']),
 '1438442': dict_keys(['status', 'bill']),
 '1588122': dict_keys(['status', 'bill']),
 '1417874': dict_keys(['status', 'bill']),
 '1398281': dict_keys(['status', 'bill']),
 '1712167': dict_keys(['status', 'bill']),
 '1581321': dict_keys(['status', 'bill']),
 '1707639': dict_keys(['status', 'bill']),
 '1592396':

In [164]:
# TEST
docs = []
docs1 = {}

for bill in bills1:
    
    # extract bill info
    bill_id = bill
    requestbill = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id="+bill_id
    b = requests.get(requestbill)
    binfo = json.loads(b.text)
    
    # FIND doc_id for most recent bill version: https://www.geeksforgeeks.org/python-check-whether-given-key-already-exists-in-a-dictionary/#
    if 'bill' in binfo.keys():
        doclist = binfo['bill']['texts']
        docs = [value['doc_id'] for value in doclist]
        docs1 = [str(doc) for doc in docs] # converting int to str
    if 'alert' in binfo.keys():
        doc_id = None
        
docs1

{'1453447': dict_keys(['status', 'bill']),
 '1693920': dict_keys(['status', 'bill']),
 '1456707': dict_keys(['status', 'bill']),
 '1388510': dict_keys(['status', 'bill']),
 '1711942': dict_keys(['status', 'bill']),
 '1454994': dict_keys(['status', 'bill']),
 '1594832': dict_keys(['status', 'bill']),
 '1614129': dict_keys(['status', 'bill']),
 '1592497': dict_keys(['status', 'bill']),
 '1453388': dict_keys(['status', 'bill']),
 '1593665': dict_keys(['status', 'bill']),
 '1702780': dict_keys(['status', 'bill']),
 '1458834': dict_keys(['status', 'bill']),
 '1433188': dict_keys(['status', 'bill']),
 '1451034': dict_keys(['status', 'bill']),
 '1578060': dict_keys(['status', 'bill']),
 '1438442': dict_keys(['status', 'bill']),
 '1588122': dict_keys(['status', 'bill']),
 '1417874': dict_keys(['status', 'bill']),
 '1398281': dict_keys(['status', 'bill']),
 '1712167': dict_keys(['status', 'bill']),
 '1581321': dict_keys(['status', 'bill']),
 '1707639': dict_keys(['status', 'bill']),
 '1592396':

In [168]:
# TEST: checking structure
docs1 = {}

for bill in bills1:
    
    # extract bill info
    bill_id = bill
    requestbill = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id="+bill_id
    b = requests.get(requestbill)
    binfo = json.loads(b.text)
    
    # FIND doc_id for most recent bill version: https://www.geeksforgeeks.org/python-check-whether-given-key-already-exists-in-a-dictionary/#
    if 'alert' in binfo.keys():
        docs1[bill] = binfo['alert'].keys()
        
docs1

{'1592779': dict_keys(['message']),
 '1592715': dict_keys(['message']),
 '1592770': dict_keys(['message']),
 '1592712': dict_keys(['message'])}

In [178]:
## EXTRACT doc_id for most recent version of bill, select bill info, save as df

# empty dict to store cleaned bill info
infotxt = {}
# empty list to store list of bill texts
texts = []

# loop
for bill in bills1:
    
    # extract bill info
    bill_id = bill
    requestbill = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id="+bill_id
    b = requests.get(requestbill)
    binfo = json.loads(b.text)
    
    # FIND doc_id for most recent bill version:
    if 'bill' in binfo.keys():
        doclist = binfo['bill']['texts']
        docs = [value['doc_id'] for value in doclist]
        docs1 = [str(doc) for doc in docs] # converting int to str
        # identify most recently amended version of doc text
        doc_id = docs1[-1] 

        # get bill text for doc text
        requestdoc = 'https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBillText&id='+doc_id
        d = requests.get(requestdoc)
        btext = json.loads(d.text)

        # get URL for doc
        URL = btext['text']['state_link']

        # get status page URL
        statusURL = btext['text']['state_link'].replace('Text', 'Status') 
        #statusURL = btext['text']['url'].split("/id")[0].replace('text', 'bill')
        URL1 = statusURL

        ## EXTRACT select info from URLs

        # main page
        r = urlopen(URL)
        html_bytes = r.read()
        html = html_bytes.decode("utf-8")
        soup = BeautifulSoup(html, features='html.parser')

        # status page
        r1 = urlopen(URL1)
        html_bytes = r1.read()
        html1 = html_bytes.decode("utf-8")
        soup1 = BeautifulSoup(html1, features='html.parser')

        # bill title
        t = soup.find_all('title')
        ti = t[0]

        # bill year
        #yi = soup.find_all(id = 'bill_intro_date') 
        yi = soup.find_all(class_ = 'bill_title_yr')

        # bill text
        txt = soup.find_all('div', id = 'bill')

        # bill status
        #status = soup1.find_all('td')
        status = soup1.find_all(id = 'billhistory')

        ## REMOVE html style and isolate key info

        # title
        for text in ti:
            title = text.text.strip()
            title = title.split("- ")[-1]

        # bill text: remove html formatting and clean
        for text in txt:
            billtxt = text.text.strip()
            # remove non ASCII chars; adapted from: https://stackoverflow.com/questions/20078816/replace-non-ascii-characters-with-a-single-space
            billtxt = re.sub(r'[^\x00-\x7F]+', ' ', billtxt)
            # remove tabs and new lines
            billtxt = re.sub(r'(\s)', ' ', billtxt)
            # add space after colons and semicolons
            billtxt = re.sub(r'(\:|\;)+', r'\1 ', billtxt)
            # add space after periods only if preceded by words >=4 chars
            billtxt = re.sub(r'([A-z]{4,}\.)+', r'\1 ', billtxt)
            # remove any extra white spaces
            billtxt = re.sub(r'\s+', ' ', billtxt)

       # year
        for text in yi:
            if text is None:
                year = None
            else:
                year = text.text.strip()
                #year = year[-4:]

       # status
        #if len(status) < 1:
         #   status = None
        #else:
            #for text in status[-1]:
        for text in status:
            status = text.text.strip()
            status = status.split("Action")[-1]
            status = re.sub(r'(\s)', ' ', status)
            status = re.sub(r'\s+', ' ', status)

    # PLACEHOLDER code until bills can be identified
    if 'alert' in binfo.keys():
        title = None
        year = None
        status = None
        billtxt = None
            
    # concatenate into one list
    infotxt = [title, year, status, billtxt]

    # add bill_id to list
    #infotxt.insert(0, bill_id)
    #bill_id = infotxt[0].split("- ")[-1][:7]
    bill_id = bill_id
    
     ## CONVERT lists to df  
    itxtdf = pd.DataFrame(infotxt)
    texts.append(itxtdf)
    fulltext = pd.concat(texts, axis = 1).transpose()

# rename columns
fulltext.columns = ['bill_id', 'title', 'year', 'status', 'text']

# inspect/show
fulltext.head()

,bill_id,title,year,status,text
0,1453447,AB-778 Institutional purchasers: purchase of C...,None,None,The people of the State of California do enact...
0,1693920,"AB-408 Climate-resilient Farms, Sustainable He...",2023,"February 02, 2023",The people of the State of California do enact...
0,1456707,AB-1009 Farm to Community Food Hub Program.,2023,None,The people of the State of California do enact...
0,1388510,"AB-125 Equitable Economic Recovery, Healthy Fo...",2020,"December 18, 2020",The people of the State of California do enact...
0,1711942,AB-1197 Agricultural Protection Planning Grant...,2023,"February 16, 2023",The people of the State of California do enact...


In [181]:
# create bill_id column after df created
#fulltext['bill_id'] = infotxt[0].split("- ")[-1][:7]
#fulltext['bill_id'] = fulltext['title']#.str.split("- ")[-1][:7]
#fulltext.head()

In [ ]:
# saving
fulltext.to_csv('CA_food and ag bills_recent.csv')

In [189]:
fulltext

,bill_id,title,year,status,text
0,1453447,AB-778 Institutional purchasers: purchase of C...,None,None,The people of the State of California do enact...
0,1693920,"AB-408 Climate-resilient Farms, Sustainable He...",2023,"February 02, 2023",The people of the State of California do enact...
0,1456707,AB-1009 Farm to Community Food Hub Program.,2023,None,The people of the State of California do enact...
0,1388510,"AB-125 Equitable Economic Recovery, Healthy Fo...",2020,"December 18, 2020",The people of the State of California do enact...
0,1711942,AB-1197 Agricultural Protection Planning Grant...,2023,"February 16, 2023",The people of the State of California do enact...
...,...,...,...,...,...
0,1398150,AB-133 Health.,2021,"[[], [], [[Procedures and]], [[Publication]], ...",The people of the State of California do enact...
0,1398254,SB-133 Budget Act of 2022.,2021,[[[\n Amended\n I...,The people of the State of California do enact...
0,1398321,SB-200 Public safety omnibus.,2021,"$ 200,000",The people of the State of California do enact...
0,1649630,AB-118 Budget Act of 2023: health.,2021,None,The people of the State of California do enact...


In [32]:
## TEST: SCRAPE SEARCH RESULTS for bill_ids: RECENT

# scrape LegiScan search results for CA bills RE: food and ag for current year
requeststring = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=2&query=food%20and%20agriculture"
# request: CA, food and ag, results for recent years
requeststring1 = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=3&query=food%20and%20agriculture"
# request: CA, food and ag, results for all available years
requeststring2 = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getSearchRaw&state=CA&year=1&query=food%20and%20agriculture"

# pull request: ge\tting results as JSON object
r = requests.get(requeststring1)
# turning JSON object into a dict
dict1 = json.loads(r.text)

# creating subset dict with only keys of interest
list1 = dict1['searchresult']['results']
# generating list of bill_ids
bills = [value['bill_id'] for value in list1]

# each bill in bills is stored as an integer; convert to list of str
bills1 = [str(bill) for bill in bills]

#inspect/show results
print(len(bills1))
print(bills1[:5])

565
['1453447', '1693920', '1456707', '1388510', '1711942']


In [4]:
# TEST CELL 
bills2 = bills1[:5]
bills2

['1453447', '1693920', '1456707', '1388510', '1711942']

In [50]:
## EXTRACT doc_id for most recent version of bill, select bill info, save as df

# empty dict to store cleaned bill info
infotxt = {}
# empty list to store list of bill texts
texts = []

# loop
for bill in bills2:
    
    # extract bill info
    bill_id = bill
    requestbill = "https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBill&id="+bill_id
    b = requests.get(requestbill)
    binfo = json.loads(b.text)
    
    # FIND doc_id for most recent bill version
    doclist = binfo['bill']['texts']
    docs = [value['doc_id'] for value in doclist]
    docs1 = [str(doc) for doc in docs] # converting int to str
    
    # identify most recently amended version of text
    doc_id = docs1[-1] 
    
    # get bill text for doc
    requestdoc = 'https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBillText&id='+doc_id
    d = requests.get(requestdoc)
    btext = json.loads(d.text)
    
    # get URL for amended doc
    URL = btext['text']['state_link']
    
    # get status page URL
    statusURL = btext['text']['state_link'].replace('Text', 'Status') 
    #statusURL = btext['text']['url'].split("/id")[0].replace('text', 'bill')
    URL1 = statusURL

    ## EXTRACT select info from URL
    
    # main page
    r = urlopen(URL)
    html_bytes = r.read()
    html = html_bytes.decode("utf-8")
    soup = BeautifulSoup(html, features='html.parser')

    # status page
    r1 = urlopen(URL1)
    html_bytes = r1.read()
    html1 = html_bytes.decode("utf-8")
    soup1 = BeautifulSoup(html1, features='html.parser')
    
    # bill title
    t = soup.find_all('title')
    ti = t[0]

    # bill year: introduced
    #yi = soup.find_all(id = 'bill_intro_date')
    yi = soup.find_all(class_ = 'bill_title_yr')

    # bill text
    txt = soup.find_all('div', id = 'bill')

    # bill status
    #status = soup.find_all('td')
    status = soup1.find_all(id = 'billhistory')
    #status = soup.find_all(id = "bill-last-action")
    
    ## REMOVE html style and tags from bill text
    
    # title
    for text in ti:
        title = text.text.strip()
        title = title.split("- ")[-1]
                  
    # bill text
    for text in txt:
        billtxt = text.text.strip()
    
    # bill status
    #if len(status) < 1:
    #    status = None
    #else:
     #   for text in status[-1]:
      #      status = text.text.strip()
            
    for text in status:
        status = text.text
        status = status.split("Action")[-1]
        status = re.sub(r'(\s)', ' ', status)
        status = re.sub(r'\s+', ' ', status)
            
        #for text in status:
         #   status = text.text.strip()
          #  status = status.split("Action: ")[-1].split("Pending")[0]
        
    # year
    for text in yi:
        if text is None:
            year = None
        else:
            year = text.text.strip()
            #year = year[-4:]
    
    # concatenating into one list
    infotxt = [title, year, status, billtxt]

    # creating bill_id
    #bill_id = infotxt[0].split("- ")[-1][:7]
    bill_id = bill_id
    
    # adding bill_id to list
    infotxt.insert(0, bill_id)
    
     ## CONVERT to df  
    itxtdf = pd.DataFrame(infotxt)
    texts.append(itxtdf)
    fulltext = pd.concat(texts, axis = 1).transpose()

# rename columns
fulltext.columns = ['bill_id', 'title', 'year', 'status', 'text']
#fulltext.columns = ['bill_id', 'title', 'status', 'text']

# inspect/show
fulltext.head()

,bill_id,title,year,status,text
0,1453447,AB-778 Institutional purchasers: purchase of C...,(2021-2022),09/27/22 Chaptered by Secretary of State - Ch...,The people of the State of California do enact...
0,1693920,"AB-408 Climate-resilient Farms, Sustainable He...",(2023-2024),"08/02/23 In committee: Set, first hearing. He...",The people of the State of California do enact...
0,1456707,AB-1009 Farm to Community Food Hub Program.,(2021-2022),10/07/21 Chaptered by Secretary of State - Ch...,The people of the State of California do enact...
0,1388510,"AB-125 Equitable Economic Recovery, Healthy Fo...",(2021-2022),02/01/22 From committee: Filed with the Chief...,The people of the State of California do enact...
0,1711942,AB-1197 Agricultural Protection Planning Grant...,(2023-2024),07/05/23 From committee: Do pass and re-refer...,The people of the State of California do enact...


In [234]:
# SAVE: pieces

# FIND doc_id for most recent bill version
doclist = binfo['bill']['texts']
docs = [value['doc_id'] for value in doclist]
docs1 = [str(doc) for doc in docs] # converting int to str

# call the most recently amended version of bill text
doc_id = docs1[-1] 

# get bill text for doc
btext1 = 'https://api.legiscan.com/?key=7e00040f1f7618af234e7415484d2494&op=getBillText&id='+doc_id
b = requests.get(btext1)
btext = json.loads(b.text)

# get URL for amended doc
URL = btext['text']['state_link']

## EXTRACTION: relevant info from URL
r = urlopen(URL)
html_bytes = r.read()
html = html_bytes.decode("utf-8")
soup = BeautifulSoup(html, features='html.parser')

# bill title
t = soup.find_all('title')
ti = t[0]

# bill year: introduced
yi = soup.find_all(id = 'bill_intro_date') # will have to be cleaned

# bill year: amended
ya = soup.find_all('tbody')
yam = (ya[0])

# bill text
txt = soup.find_all('div', id = 'bill')

## REMOVE: html style and tags from bill text

# title
for text in ti:
    title = text.text.strip()
    title = title.split("- ")[-1]

# year intro
for text in yi:
    yeari = text.text.strip()
    yeari = yeari[-4:]

# year recent activity
for text in yam:
    yeara = text.text.strip()
    yeara = yeara[-4:]

# bill text
for text in txt:
    billtxt = text.text.strip()
    
# creating bill_id
bill_id = title.split("- ")[-1][:7]

# concatenating into one list
infotxt = [bill_id, title, yeari, yeara, billtxt]
infotxt

## CONVERT to df

itxtdf = pd.DataFrame(infotxt).transpose()

# rename columns
itxtdf.columns = ['bill_id', 'title', 'y_intro', 'y_recent', 'text']
itxtdf

,status,bill
text,OK,"{'bill_id': 1711942, 'change_hash': 'cd9d825f6..."
text,OK,"{'bill_id': 1693920, 'change_hash': '464be2cde..."
text,OK,"{'bill_id': 1702780, 'change_hash': '266b25df3..."
text,OK,"{'bill_id': 1712167, 'change_hash': '37ed87e95..."
text,OK,"{'bill_id': 1707639, 'change_hash': '6bddb0f6f..."
text,OK,"{'bill_id': 1671132, 'change_hash': '3f4199a2f..."
text,OK,"{'bill_id': 1712103, 'change_hash': 'ce1242ff4..."
text,OK,"{'bill_id': 1707512, 'change_hash': '85d63281d..."
text,OK,"{'bill_id': 1714501, 'change_hash': 'f094de3f6..."
text,OK,"{'bill_id': 1738922, 'change_hash': '95e252ff3..."
